In [7]:
import numpy as np
import datetime as dt
import math
from minilp import *
import minilp.result
from Multiple_knapsack import Multiple_knapsack

In [8]:


def get_first_non_integral(p, s):
    """ Given a problem p and a result s, returns the first integer variables
    that has a fractional value in s, or None if no such variable was found. """
    for v in p.variables:
        if v.category == int:
            if abs(round(s.get_value(v)) - s.get_value(v)) > 1e-6:
                return v
    return None

def banb(p, lp_solver=None):
    if lp_solver is None:
        lp_solver = solvers.get_default_solver()
    def cmp(l, r):
        if np.isnan(r.objective):
            return True
        if p.sense == min:
            return l.objective < r.objective
        return l.objective > r.objective

    res = minilp.result.result()
    #res = p.lp_solve(lp_solver)
    nodes = [[]] # nodes = list of constraints
    cnt = 0
    
    print('B&B using {} to solve linear relaxation'.format(lp_solver.__class__.__name__))
    while nodes:
        if cnt % 10 == 0:
            print('{} {:5d} {:5d} {:9g}'.format(dt.datetime.now().strftime('%T'), cnt, 
                                                len(nodes), res.objective))
        cnt += 1
        cns = nodes.pop()
        cs = p.add_constraints(cns)
        s = p.lp_solve(lp_solver)
        p.del_constraints(cs)
        if s:
            fni = get_first_non_integral(p, s)
            if fni is None: # integral solution
                if cmp(s, res):
                    res = s
                    print('{} {:5d} {:5d} {:9g}*'.format(dt.datetime.now().strftime('%T'), cnt, 
                                                         len(nodes), res.objective))
            elif cmp(s, res):
                fl = math.floor(s.get_value(fni))
                nodes.append(cns + [fni <= fl])
                nodes.append(cns + [fni >= fl + 1])
    return res

In [9]:
from Multiple_knapsack import Multiple_knapsack

pb = Multiple_knapsack("Sources_Files/dc2.in")
a = pb.servers
pb.fileToProblem()
s = pb.flat()
c = []
z = []
servers = pb.servers
nb_servers = len(servers)
nb_sacs = len(s)
for server in servers:
    z.append(server[0])
    c.append(server[1])
print(s)

[7, 8, 8, 8]


In [10]:
c

[10, 10, 5, 5, 1, 50, 50, 100, 100, 100]

In [15]:
lp = problem('PLNE')

#existence of a server i in a line j
e = np.array(lp.integer_var_list(nb_sacs*nb_servers,0,1)).reshape(nb_sacs,nb_servers)

#a server appears only one time
for i in range(nb_servers):
    lp.add_constraint(sum(e.T[i]) <= 1)

for j in range(nb_sacs):
    lp.add_constraint( sum(e[j]*z) <= s[j] )

lp.set_objective( sum(sum(e*c)), sense=max)

result=banb(lp)
print(result)

B&B using docplex to solve linear relaxation
11:57:50     0     1       nan
11:57:51    10     3       nan
11:57:51    19     4       325*
11:57:51    20     3       325
11:57:51    30     3       325
11:57:51    40     3       325
11:57:52    50     3       325
11:57:52    60     3       325
11:57:52    70     1       325
11:57:52    80     3       325
11:57:53    90     3       325
11:57:53   100     3       325
11:57:53   101     2       325*
11:57:53   110     3       325
11:57:53   120     3       325
status = optimal, obj. = 325.00000000000006


In [23]:
res_e = []
for j in range(nb_sacs):
    res_e.append(result.get_values(e[j]))
    print(res_e[j])

[1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0]
[0, 1.0, 0, 1.0, 0, 0, 1.0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0]


In [11]:


class PLNE():
    def __init__(self):
        self.flat = 0                # sizes of knapsacks 
        self.nb_knapsack = 0         # amount of knapsack
        self.servers = 0             # list of servers
        self.nb_servers = 0          # amount of servers
        self.existence_servers = []  # existence of a server j in knapsack i
        self.sizes_server = []       # size of servers
        self.capacities_server = []  # calculating capacity of servers
        self.result = []             # final result of existence_servers
        self.time_consumption = None # time comsumption of PLNE method
        
    def read_file(self, path):
        #read file and import variable values, more details are in Multiple_knapsack.py
        pb = Multiple_knapsack(path)         
        self.servers = pb.servers
        self.flat = pb.flat()
        self.nb_knapsack = len(pb.flat())
        self.nb_servers = len(self.servers)
        for server in self.servers:
            self.sizes_server.append(server[0])
            self.capacities_server.append(server[1])
            
    def solve(self):
        #solve the problem
        start_time = time.time()
        lp = problem('PLNE')
        self.existence_servers = np.array(lp.integer_var_list(self.nb_knapsack*self.nb_servers,0,1)) \
                                                        .reshape(self.nb_knapsack,self.nb_servers)
        # a server exists only one time
        for i in range(self.nb_servers):
            lp.add_constraint(sum(self.existence_servers.T[i]) <= 1)
        # each sac has limited size
        for j in range(self.nb_knapsack):
            lp.add_constraint( sum(self.existence_servers[j]*self.sizes_server) <= self.flat[j] )
        # set objective function to be maximized
        lp.set_objective( sum(sum(self.existence_servers*self.capacities_server)), sense=max)
        result=banb(lp)
        
        self.time_consumption = time.time() - start_time
        # get the existence of servers in each knapsack
        for j in range(self.nb_knapsack):
            self.result.append(result.get_values(self.existence_servers[j]))
        print("OK, problem is solved within %.2f seconds" %(self.time_consumption))

In [12]:
aaa=PLNE()
aaa.read_file("Sources_Files/dc2.in")
aaa.solve()
aaa.result

NameError: name 'banb' is not defined

In [6]:
help(banb)

Help on function banb in module __main__:

banb(p, lp_solver=None)

